In [1]:
import pandas as pd
import numpy as np


# for Modelling
from sklearn.preprocessing import MinMaxScaler
from tensorflow.keras.models import Sequential
from tensorflow.keras.layers import SimpleRNN, GRU, LSTM, Dense

from tensorflow.keras.models import load_model

# for Evaluation
from sklearn.metrics import r2_score, mean_absolute_error, mean_squared_error
from sklearn.metrics import mean_absolute_error, r2_score


In [2]:
df= pd.read_csv('bx_t1.csv')
df

,Time id,Material,Quantity,Total_Value,Material_Id
0,1,C01-100100,4.0,11560.00,108
1,1,C06-501311,31.0,36731.17,719
2,1,C06-700217,2.0,25900.00,797
3,1,C07-500328,22.0,44702.25,884
4,1,C37-000235,7.0,23828.07,1663
...,...,...,...,...,...
890,38,S12-160336,3.0,39648.85,14447
891,38,S12-160337,3.0,45048.86,14448
892,38,S23-100109,13.0,71666.86,8594
893,38,S31-020101,317.0,65936.00,9207


In [3]:
import pandas as pd

# Assuming your DataFrame is named df

# Filter the DataFrame for Time ids 37 and 38
filtered_df = df[df['Time id'].isin([37, 38])]

# Display the resulting DataFrame
print(filtered_df)


     Time id       Material  Quantity  Total_Value  Material_Id
848       37     C01-100100      56.0    155495.42          108
849       37     C06-501311      18.0     28440.00          719
850       37     C06-700217       6.0     82542.24          797
851       37     C07-500328      26.0     68203.77          884
852       37     C37-000235      30.0    121504.33         1663
853       37     C37-000236      27.0    117449.53         1664
854       37     C37-200170       2.0     46512.50         1692
855       37     C37-300106       8.0     29244.27         1715
856       37     C37-300108       8.0     24482.08         1716
857       37     C37-400135       3.0    133629.39         1732
858       37     C72-100398       1.0     70700.00         2600
859       37     C77-401591     108.0    139131.58         2865
860       37     C77-401592     108.0    121216.18         2866
861       37     C78-105698      90.0     57565.86         3282
862       37     C97-000204      41.0   

In [4]:
from sklearn.metrics import mean_absolute_error, r2_score
from sklearn.preprocessing import MinMaxScaler
from keras.models import Sequential
from keras.layers import SimpleRNN, GRU, LSTM, Dense
import numpy as np
import pandas as pd

# Define the evaluate_model function with MAPE
def evaluate_model(model, X_test, y_test):
    y_pred = model.predict(X_test)
    mae = mean_absolute_error(y_test, y_pred)
    mape = np.mean(np.abs((y_test - y_pred) / y_test)) * 100
    r2 = r2_score(y_test, y_pred)
    return mae, mape, r2

# Define train and test periods
train_start_id = 25
train_end_id = 36
forecast_time_id = 37

# Create an empty DataFrame to store forecasted values
forecast_df = pd.DataFrame(columns=['Time id', 'Material', 'Quantity', 'Total_Value', 'Material_Id', 'Model', 'Forecasted', 'MAE', 'MAPE', 'R2'])

# Define a function to build and train RNN model
def build_and_train_model(X_train, y_train, model_type):
    model = Sequential()
    if model_type == 'SimpleRNN':
        model.add(SimpleRNN(units=64, input_shape=(X_train.shape[1], X_train.shape[2])))
    elif model_type == 'GRU':
        model.add(GRU(units=64, input_shape=(X_train.shape[1], X_train.shape[2])))
    elif model_type == 'LSTM':
        model.add(LSTM(units=64, input_shape=(X_train.shape[1], X_train.shape[2])))
    model.add(Dense(1))
    model.compile(optimizer='adam', loss='mse')
    model.fit(X_train, y_train, epochs=100, batch_size=32, verbose=0)
    return model

# Iterate over each material id
for material_id in df['Material_Id'].unique():
    print(f"Evaluating models for Material ID: {material_id}")
    
    # Filter data for the current material id
    material_data = df[df['Material_Id'] == material_id]
    
    # Filter training data
    # Filter training data
    train_data = material_data[(material_data['Time id'] >= train_start_id) & (material_data['Time id'] <= train_end_id)]


    # Skip evaluation if there is no training data
    if train_data.empty:
        print(f"No training data available for Material ID: {material_id}. Skipping evaluation.")
        continue

    # Scale the data
    scaler = MinMaxScaler()
    train_scaled = scaler.fit_transform(train_data[['Quantity']])

    # Reshape input to be [samples, time steps, features]
    X_train = np.reshape(train_scaled, (train_scaled.shape[0], 1, 1))
    y_train = train_scaled[:, 0]  # Target variable is Quantity

    # Evaluate SimpleRNN, GRU, and LSTM models
    for model_type in ['SimpleRNN', 'GRU', 'LSTM']:
        print(f"Training {model_type} model for Material ID: {material_id}")
        
        # Build and train the model
        model = build_and_train_model(X_train, y_train, model_type)
        
        # Forecast for the next time id (Time id 37)
        forecasted_values = model.predict(X_train[-1:])  # Use the last time step of the training data for forecasting
        forecasted_values = scaler.inverse_transform(forecasted_values.reshape(-1, 1)).flatten()
        forecast = forecasted_values[0]

        # Evaluate the model for the current Material Id
        mae, mape, r2 = evaluate_model(model, X_train, y_train)
        print(f"MAE: {mae}, MAPE: {mape}, R-squared: {r2}")

        # Append to forecast DataFrame
        forecast_df = forecast_df.append({
            'Time id': forecast_time_id,
            'Material': material_data['Material'].iloc[-1],
            'Quantity': material_data['Quantity'].iloc[-1],
            'Total_Value': material_data['Total_Value'].iloc[-1],
            'Material_Id': material_id,
            'Model': model_type,
            'Forecasted': forecast,
            'MAE': mae,
            'MAPE': mape,
            'R2': r2
        }, ignore_index=True)

# Display the forecast DataFrame
print(forecast_df)


Evaluating models for Material ID: 108
Training SimpleRNN model for Material ID: 108


C:\Users\sohan\anaconda3\lib\site-packages\keras\src\layers\rnn\rnn.py:205: UserWarning: Do not pass an `input_shape`/`input_dim` argument to a layer. When using Sequential models, prefer using an `Input(shape)` object as the first layer in the model instead.
  super().__init__(**kwargs)


1/1 ━━━━━━━━━━━━━━━━━━━━ 0s 68ms/step
1/1 ━━━━━━━━━━━━━━━━━━━━ 0s 73ms/step
MAE: 0.027012431949731852, MAPE: inf, R-squared: 0.9885164227206285
Training GRU model for Material ID: 108


C:\Users\sohan\AppData\Local\Temp\ipykernel_24440\4108763989.py:12: RuntimeWarning: divide by zero encountered in divide
  mape = np.mean(np.abs((y_test - y_pred) / y_test)) * 100
C:\Users\sohan\AppData\Local\Temp\ipykernel_24440\4108763989.py:80: FutureWarning: The frame.append method is deprecated and will be removed from pandas in a future version. Use pandas.concat instead.
  forecast_df = forecast_df.append({
C:\Users\sohan\anaconda3\lib\site-packages\keras\src\layers\rnn\rnn.py:205: UserWarning: Do not pass an `input_shape`/`input_dim` argument to a layer. When using Sequential models, prefer using an `Input(shape)` object as the first layer in the model instead.
  super().__init__(**kwargs)


1/1 ━━━━━━━━━━━━━━━━━━━━ 0s 223ms/step
1/1 ━━━━━━━━━━━━━━━━━━━━ 0s 285ms/step
MAE: 0.08894996806269599, MAPE: inf, R-squared: 0.8685151834044144
Training LSTM model for Material ID: 108


...

C:\Users\sohan\AppData\Local\Temp\ipykernel_24440\4108763989.py:12: RuntimeWarning: divide by zero encountered in divide
  mape = np.mean(np.abs((y_test - y_pred) / y_test)) * 100
C:\Users\sohan\AppData\Local\Temp\ipykernel_24440\4108763989.py:80: FutureWarning: The frame.append method is deprecated and will be removed from pandas in a future version. Use pandas.concat instead.
  forecast_df = forecast_df.append({
C:\Users\sohan\anaconda3\lib\site-packages\keras\src\layers\rnn\rnn.py:205: UserWarning: Do not pass an `input_shape`/`input_dim` argument to a layer. When using Sequential models, prefer using an `Input(shape)` object as the first layer in the model instead.
  super().__init__(**kwargs)


1/1 ━━━━━━━━━━━━━━━━━━━━ 0s 245ms/step
1/1 ━━━━━━━━━━━━━━━━━━━━ 0s 266ms/step
MAE: 0.07872754111886027, MAPE: inf, R-squared: 0.9080273305435025
Training LSTM model for Material ID: 14448


C:\Users\sohan\AppData\Local\Temp\ipykernel_24440\4108763989.py:12: RuntimeWarning: divide by zero encountered in divide
  mape = np.mean(np.abs((y_test - y_pred) / y_test)) * 100
C:\Users\sohan\AppData\Local\Temp\ipykernel_24440\4108763989.py:80: FutureWarning: The frame.append method is deprecated and will be removed from pandas in a future version. Use pandas.concat instead.
  forecast_df = forecast_df.append({
C:\Users\sohan\anaconda3\lib\site-packages\keras\src\layers\rnn\rnn.py:205: UserWarning: Do not pass an `input_shape`/`input_dim` argument to a layer. When using Sequential models, prefer using an `Input(shape)` object as the first layer in the model instead.
  super().__init__(**kwargs)


1/1 ━━━━━━━━━━━━━━━━━━━━ 0s 209ms/step
1/1 ━━━━━━━━━━━━━━━━━━━━ 0s 211ms/step
MAE: 0.15695448865493142, MAPE: inf, R-squared: 0.6462315882454448
   Time id    Material  Quantity  Total_Value Material_Id      Model  \
0       37  C01-100100      48.0    133276.00         108  SimpleRNN   
1       37  C01-100100      48.0    133276.00         108        GRU   
2       37  C01-100100      48.0    133276.00         108       LSTM   
3       37  C06-501311      28.0     44240.00         719  SimpleRNN   
4       37  C06-501311      28.0     44240.00         719        GRU   
..     ...         ...       ...          ...         ...        ...   
70      37  S12-160336       3.0     39648.85       14447        GRU   
71      37  S12-160336       3.0     39648.85       14447       LSTM   
72      37  S12-160337       3.0     45048.86       14448  SimpleRNN   
73      37  S12-160337       3.0     45048.86       14448        GRU   
74      37  S12-160337       3.0     45048.86       14448      

C:\Users\sohan\AppData\Local\Temp\ipykernel_24440\4108763989.py:12: RuntimeWarning: divide by zero encountered in divide
  mape = np.mean(np.abs((y_test - y_pred) / y_test)) * 100
C:\Users\sohan\AppData\Local\Temp\ipykernel_24440\4108763989.py:80: FutureWarning: The frame.append method is deprecated and will be removed from pandas in a future version. Use pandas.concat instead.
  forecast_df = forecast_df.append({


In [5]:


# Group the forecast DataFrame by the 'Model' column and calculate the mean MAE and R2 for each group
model_metrics = forecast_df.groupby('Model').agg({'MAE': 'mean', 'R2': 'mean'}).reset_index()

# Print the overall MAE and R2 for each model
print("Overall MAE and R2 for each model:")
print(model_metrics)

Overall MAE and R2 for each model:
       Model       MAE        R2
0        GRU  0.109803  0.792429
1       LSTM  0.151389  0.642151
2  SimpleRNN  0.056036  0.918971


In [6]:
from sklearn.preprocessing import MinMaxScaler
from keras.models import Sequential
from keras.layers import SimpleRNN, GRU, LSTM, Dense
import numpy as np
import pandas as pd

# Define the evaluate_model function with MAPE
def evaluate_model(y_pred, y_actual):
    mape = np.mean(np.abs((y_actual - y_pred) / y_actual)) * 100
    return mape

# Define train and test periods
train_start_id = 25
train_end_id = 36
forecast_time_id = 37

# Create an empty DataFrame to store forecasted values
forecast_df = pd.DataFrame(columns=['Material_Id', 'Model', 'MAPE'])

# Define a function to build and train RNN model
def build_and_train_model(X_train, y_train, model_type):
    model = Sequential()
    if model_type == 'SimpleRNN':
        model.add(SimpleRNN(units=64, input_shape=(X_train.shape[1], X_train.shape[2])))
    elif model_type == 'GRU':
        model.add(GRU(units=64, input_shape=(X_train.shape[1], X_train.shape[2])))
    elif model_type == 'LSTM':
        model.add(LSTM(units=64, input_shape=(X_train.shape[1], X_train.shape[2])))
    model.add(Dense(1))
    model.compile(optimizer='adam', loss='mse')
    model.fit(X_train, y_train, epochs=100, batch_size=32, verbose=0)
    return model

# Iterate over each material id
for material_id in df['Material_Id'].unique():
    print(f"Evaluating models for Material ID: {material_id}")
    
    # Filter data for the current material id
    material_data = df[df['Material_Id'] == material_id]
    
    # Filter training data
    train_data = material_data[(material_data['Time id'] >= train_start_id) & (material_data['Time id'] <= train_end_id)]
    test_data = material_data[material_data['Time id'] == forecast_time_id]

    # Skip evaluation if there is no training or test data
    if train_data.empty or test_data.empty:
        print(f"No training or test data available for Material ID: {material_id}. Skipping evaluation.")
        continue

    # Scale the data
    scaler = MinMaxScaler()
    train_scaled = scaler.fit_transform(train_data[['Quantity']])

    # Reshape input to be [samples, time steps, features]
    X_train = np.reshape(train_scaled, (train_scaled.shape[0], 1, 1))
    y_train = train_scaled[:, 0]  # Target variable is Quantity

    # Evaluate SimpleRNN, GRU, and LSTM models
    for model_type in ['SimpleRNN', 'GRU', 'LSTM']:
        print(f"Evaluating {model_type} model for Material ID: {material_id}")

        # Build and train the model
        model = build_and_train_model(X_train, y_train, model_type)

        # Forecast for the next time id (Time id 37)
        forecast_scaled = model.predict(X_train[-1:])  # Use the last time step of the training data for forecasting
        forecast = scaler.inverse_transform(forecast_scaled.reshape(-1, 1)).flatten()[0]

        # Get actual Quantity value for Time id 37
        actual_quantity = test_data['Quantity'].values[0]

        # Evaluate the model for the current Material Id
        mape = evaluate_model(np.array([forecast]), np.array([actual_quantity]))
        print(f"MAPE: {mape}")

        # Append to forecast DataFrame
        forecast_df = forecast_df.append({
            'Material_Id': material_id,
            'Model': model_type,
            'MAPE': mape
        }, ignore_index=True)

# Display the forecast DataFrame with MAPE values
print(forecast_df)


Evaluating models for Material ID: 108
Evaluating SimpleRNN model for Material ID: 108


C:\Users\sohan\anaconda3\lib\site-packages\keras\src\layers\rnn\rnn.py:205: UserWarning: Do not pass an `input_shape`/`input_dim` argument to a layer. When using Sequential models, prefer using an `Input(shape)` object as the first layer in the model instead.
  super().__init__(**kwargs)


1/1 ━━━━━━━━━━━━━━━━━━━━ 0s 193ms/step
MAPE: 42.32728140694754
Evaluating GRU model for Material ID: 108


C:\Users\sohan\AppData\Local\Temp\ipykernel_24440\472838995.py:77: FutureWarning: The frame.append method is deprecated and will be removed from pandas in a future version. Use pandas.concat instead.
  forecast_df = forecast_df.append({
C:\Users\sohan\anaconda3\lib\site-packages\keras\src\layers\rnn\rnn.py:205: UserWarning: Do not pass an `input_shape`/`input_dim` argument to a layer. When using Sequential models, prefer using an `Input(shape)` object as the first layer in the model instead.
  super().__init__(**kwargs)


1/1 ━━━━━━━━━━━━━━━━━━━━ 0s 249ms/step
MAPE: 48.1504099709647
Evaluating LSTM model for Material ID: 108


...

C:\Users\sohan\AppData\Local\Temp\ipykernel_24440\472838995.py:77: FutureWarning: The frame.append method is deprecated and will be removed from pandas in a future version. Use pandas.concat instead.
  forecast_df = forecast_df.append({
C:\Users\sohan\anaconda3\lib\site-packages\keras\src\layers\rnn\rnn.py:205: UserWarning: Do not pass an `input_shape`/`input_dim` argument to a layer. When using Sequential models, prefer using an `Input(shape)` object as the first layer in the model instead.
  super().__init__(**kwargs)


1/1 ━━━━━━━━━━━━━━━━━━━━ 0s 249ms/step
MAPE: 297.13237285614014
Evaluating LSTM model for Material ID: 13739


C:\Users\sohan\AppData\Local\Temp\ipykernel_24440\472838995.py:77: FutureWarning: The frame.append method is deprecated and will be removed from pandas in a future version. Use pandas.concat instead.
  forecast_df = forecast_df.append({
C:\Users\sohan\anaconda3\lib\site-packages\keras\src\layers\rnn\rnn.py:205: UserWarning: Do not pass an `input_shape`/`input_dim` argument to a layer. When using Sequential models, prefer using an `Input(shape)` object as the first layer in the model instead.
  super().__init__(**kwargs)


1/1 ━━━━━━━━━━━━━━━━━━━━ 0s 206ms/step
MAPE: 347.1477508544922
Evaluating models for Material ID: 14447
No training or test data available for Material ID: 14447. Skipping evaluation.
Evaluating models for Material ID: 14448
No training or test data available for Material ID: 14448. Skipping evaluation.
   Material_Id      Model        MAPE
0          108  SimpleRNN   42.327281
1          108        GRU   48.150410
2          108       LSTM   51.805326
3          719  SimpleRNN   34.756131
4          719        GRU   48.255359
..         ...        ...         ...
64        9207        GRU   15.774986
65        9207       LSTM   15.311809
66       13739  SimpleRNN  237.791491
67       13739        GRU  297.132373
68       13739       LSTM  347.147751

[69 rows x 3 columns]


C:\Users\sohan\AppData\Local\Temp\ipykernel_24440\472838995.py:77: FutureWarning: The frame.append method is deprecated and will be removed from pandas in a future version. Use pandas.concat instead.
  forecast_df = forecast_df.append({


In [7]:

# Calculate the overall MAPE for each model type
overall_mape = forecast_df.groupby('Model')['MAPE'].mean().reset_index()

# Print the overall MAPE for each model type
print("Overall MAPE for each model type:")
print(overall_mape)


Overall MAPE for each model type:
       Model       MAPE
0        GRU  71.766867
1       LSTM  72.261853
2  SimpleRNN  70.509565


In [8]:
from sklearn.metrics import mean_absolute_error, r2_score
from sklearn.preprocessing import MinMaxScaler
from keras.models import Sequential
from keras.layers import SimpleRNN, GRU, LSTM, Dense
import numpy as np
import pandas as pd

# Define the evaluate_model function with MAPE
def evaluate_model(model, X_test, y_test):
    y_pred = model.predict(X_test)
    mae = mean_absolute_error(y_test, y_pred)
    mape = np.mean(np.abs((y_test - y_pred) / y_test)) * 100
    r2 = r2_score(y_test, y_pred)
    return mae, mape, r2

# Define train and test periods
train_start_id = 25
train_end_id = 37
forecast_time_id = 38

# Create an empty DataFrame to store forecasted values
forecast_df = pd.DataFrame(columns=['Time id', 'Material', 'Quantity', 'Total_Value', 'Material_Id', 'Model', 'Forecasted', 'MAE', 'MAPE', 'R2'])

# Define a function to build and train RNN model
def build_and_train_model(X_train, y_train, model_type):
    model = Sequential()
    if model_type == 'SimpleRNN':
        model.add(SimpleRNN(units=64, input_shape=(X_train.shape[1], X_train.shape[2])))
    elif model_type == 'GRU':
        model.add(GRU(units=64, input_shape=(X_train.shape[1], X_train.shape[2])))
    elif model_type == 'LSTM':
        model.add(LSTM(units=64, input_shape=(X_train.shape[1], X_train.shape[2])))
    model.add(Dense(1))
    model.compile(optimizer='adam', loss='mse')
    model.fit(X_train, y_train, epochs=100, batch_size=32, verbose=0)
    return model

# Iterate over each material id
for material_id in df['Material_Id'].unique():
    print(f"Evaluating models for Material ID: {material_id}")
    
    # Filter data for the current material id
    material_data = df[df['Material_Id'] == material_id]
    
    # Filter training data
    # Filter training data
    train_data = material_data[(material_data['Time id'] >= train_start_id) & (material_data['Time id'] <= train_end_id)]


    # Skip evaluation if there is no training data
    if train_data.empty:
        print(f"No training data available for Material ID: {material_id}. Skipping evaluation.")
        continue

    # Scale the data
    scaler = MinMaxScaler()
    train_scaled = scaler.fit_transform(train_data[['Quantity']])

    # Reshape input to be [samples, time steps, features]
    X_train = np.reshape(train_scaled, (train_scaled.shape[0], 1, 1))
    y_train = train_scaled[:, 0]  # Target variable is Quantity

    # Evaluate SimpleRNN, GRU, and LSTM models
    for model_type in ['SimpleRNN', 'GRU', 'LSTM']:
        print(f"Training {model_type} model for Material ID: {material_id}")
        
        # Build and train the model
        model = build_and_train_model(X_train, y_train, model_type)
        
        # Forecast for the next time id (Time id 37)
        forecasted_values = model.predict(X_train[-1:])  # Use the last time step of the training data for forecasting
        forecasted_values = scaler.inverse_transform(forecasted_values.reshape(-1, 1)).flatten()
        forecast = forecasted_values[0]

        # Evaluate the model for the current Material Id
        mae, mape, r2 = evaluate_model(model, X_train, y_train)
        print(f"MAE: {mae}, MAPE: {mape}, R-squared: {r2}")

        # Append to forecast DataFrame
        forecast_df = forecast_df.append({
            'Time id': forecast_time_id,
            'Material': material_data['Material'].iloc[-1],
            'Quantity': material_data['Quantity'].iloc[-1],
            'Total_Value': material_data['Total_Value'].iloc[-1],
            'Material_Id': material_id,
            'Model': model_type,
            'Forecasted': forecast,
            'MAE': mae,
            'MAPE': mape,
            'R2': r2
        }, ignore_index=True)

# Display the forecast DataFrame
print(forecast_df)


Evaluating models for Material ID: 108
Training SimpleRNN model for Material ID: 108


C:\Users\sohan\anaconda3\lib\site-packages\keras\src\layers\rnn\rnn.py:205: UserWarning: Do not pass an `input_shape`/`input_dim` argument to a layer. When using Sequential models, prefer using an `Input(shape)` object as the first layer in the model instead.
  super().__init__(**kwargs)


1/1 ━━━━━━━━━━━━━━━━━━━━ 0s 169ms/step
1/1 ━━━━━━━━━━━━━━━━━━━━ 0s 193ms/step
MAE: 0.03832701479013149, MAPE: inf, R-squared: 0.9811754302016422
Training GRU model for Material ID: 108


C:\Users\sohan\AppData\Local\Temp\ipykernel_24440\3232560010.py:12: RuntimeWarning: divide by zero encountered in divide
  mape = np.mean(np.abs((y_test - y_pred) / y_test)) * 100
C:\Users\sohan\AppData\Local\Temp\ipykernel_24440\3232560010.py:80: FutureWarning: The frame.append method is deprecated and will be removed from pandas in a future version. Use pandas.concat instead.
  forecast_df = forecast_df.append({
C:\Users\sohan\anaconda3\lib\site-packages\keras\src\layers\rnn\rnn.py:205: UserWarning: Do not pass an `input_shape`/`input_dim` argument to a layer. When using Sequential models, prefer using an `Input(shape)` object as the first layer in the model instead.
  super().__init__(**kwargs)


1/1 ━━━━━━━━━━━━━━━━━━━━ 0s 332ms/step
1/1 ━━━━━━━━━━━━━━━━━━━━ 0s 276ms/step
MAE: 0.1224100051017908, MAPE: inf, R-squared: 0.800662895716267
Training LSTM model for Material ID: 108


...

C:\Users\sohan\AppData\Local\Temp\ipykernel_24440\3232560010.py:12: RuntimeWarning: divide by zero encountered in divide
  mape = np.mean(np.abs((y_test - y_pred) / y_test)) * 100
C:\Users\sohan\AppData\Local\Temp\ipykernel_24440\3232560010.py:80: FutureWarning: The frame.append method is deprecated and will be removed from pandas in a future version. Use pandas.concat instead.
  forecast_df = forecast_df.append({
C:\Users\sohan\anaconda3\lib\site-packages\keras\src\layers\rnn\rnn.py:205: UserWarning: Do not pass an `input_shape`/`input_dim` argument to a layer. When using Sequential models, prefer using an `Input(shape)` object as the first layer in the model instead.
  super().__init__(**kwargs)


1/1 ━━━━━━━━━━━━━━━━━━━━ 0s 254ms/step
1/1 ━━━━━━━━━━━━━━━━━━━━ 0s 287ms/step
MAE: 0.0749342444042365, MAPE: inf, R-squared: 0.9166716415499161
Training LSTM model for Material ID: 14448


C:\Users\sohan\AppData\Local\Temp\ipykernel_24440\3232560010.py:12: RuntimeWarning: divide by zero encountered in divide
  mape = np.mean(np.abs((y_test - y_pred) / y_test)) * 100
C:\Users\sohan\AppData\Local\Temp\ipykernel_24440\3232560010.py:80: FutureWarning: The frame.append method is deprecated and will be removed from pandas in a future version. Use pandas.concat instead.
  forecast_df = forecast_df.append({
C:\Users\sohan\anaconda3\lib\site-packages\keras\src\layers\rnn\rnn.py:205: UserWarning: Do not pass an `input_shape`/`input_dim` argument to a layer. When using Sequential models, prefer using an `Input(shape)` object as the first layer in the model instead.
  super().__init__(**kwargs)


1/1 ━━━━━━━━━━━━━━━━━━━━ 0s 251ms/step
1/1 ━━━━━━━━━━━━━━━━━━━━ 0s 273ms/step
MAE: 0.1550451308488846, MAPE: inf, R-squared: 0.656705757343547
   Time id    Material  Quantity  Total_Value Material_Id      Model  \
0       38  C01-100100      48.0    133276.00         108  SimpleRNN   
1       38  C01-100100      48.0    133276.00         108        GRU   
2       38  C01-100100      48.0    133276.00         108       LSTM   
3       38  C06-501311      28.0     44240.00         719  SimpleRNN   
4       38  C06-501311      28.0     44240.00         719        GRU   
..     ...         ...       ...          ...         ...        ...   
70      38  S12-160336       3.0     39648.85       14447        GRU   
71      38  S12-160336       3.0     39648.85       14447       LSTM   
72      38  S12-160337       3.0     45048.86       14448  SimpleRNN   
73      38  S12-160337       3.0     45048.86       14448        GRU   
74      38  S12-160337       3.0     45048.86       14448       L

C:\Users\sohan\AppData\Local\Temp\ipykernel_24440\3232560010.py:12: RuntimeWarning: divide by zero encountered in divide
  mape = np.mean(np.abs((y_test - y_pred) / y_test)) * 100
C:\Users\sohan\AppData\Local\Temp\ipykernel_24440\3232560010.py:80: FutureWarning: The frame.append method is deprecated and will be removed from pandas in a future version. Use pandas.concat instead.
  forecast_df = forecast_df.append({


In [9]:
# Group the forecast DataFrame by the 'Model' column and calculate the mean MAE and R2 for each group
model_metrics = forecast_df.groupby('Model').agg({'MAE': 'mean', 'R2': 'mean'}).reset_index()

# Print the overall MAE and R2 for each model
print("Overall MAE and R2 for each model:")
print(model_metrics)


Overall MAE and R2 for each model:
       Model       MAE        R2
0        GRU  0.113933  0.760718
1       LSTM  0.146267  0.635172
2  SimpleRNN  0.050218  0.926898


In [10]:
from sklearn.preprocessing import MinMaxScaler
from keras.models import Sequential
from keras.layers import SimpleRNN, GRU, LSTM, Dense
import numpy as np
import pandas as pd

# Define the evaluate_model function with MAPE
def evaluate_model(y_pred, y_actual):
    mape = np.mean(np.abs((y_actual - y_pred) / y_actual)) * 100
    return mape

# Define train and test periods
train_start_id = 25
train_end_id = 37
forecast_time_id = 38

# Create an empty DataFrame to store forecasted values
forecast_df = pd.DataFrame(columns=['Material_Id', 'Model', 'MAPE'])

# Define a function to build and train RNN model
def build_and_train_model(X_train, y_train, model_type):
    model = Sequential()
    if model_type == 'SimpleRNN':
        model.add(SimpleRNN(units=64, input_shape=(X_train.shape[1], X_train.shape[2])))
    elif model_type == 'GRU':
        model.add(GRU(units=64, input_shape=(X_train.shape[1], X_train.shape[2])))
    elif model_type == 'LSTM':
        model.add(LSTM(units=64, input_shape=(X_train.shape[1], X_train.shape[2])))
    model.add(Dense(1))
    model.compile(optimizer='adam', loss='mse')
    model.fit(X_train, y_train, epochs=100, batch_size=32, verbose=0)
    return model

# Iterate over each material id
for material_id in df['Material_Id'].unique():
    print(f"Evaluating models for Material ID: {material_id}")
    
    # Filter data for the current material id
    material_data = df[df['Material_Id'] == material_id]
    
    # Filter training data
    train_data = material_data[(material_data['Time id'] >= train_start_id) & (material_data['Time id'] <= train_end_id)]
    test_data = material_data[material_data['Time id'] == forecast_time_id]

    # Skip evaluation if there is no training or test data
    if train_data.empty or test_data.empty:
        print(f"No training or test data available for Material ID: {material_id}. Skipping evaluation.")
        continue

    # Scale the data
    scaler = MinMaxScaler()
    train_scaled = scaler.fit_transform(train_data[['Quantity']])

    # Reshape input to be [samples, time steps, features]
    X_train = np.reshape(train_scaled, (train_scaled.shape[0], 1, 1))
    y_train = train_scaled[:, 0]  # Target variable is Quantity

    # Evaluate SimpleRNN, GRU, and LSTM models
    for model_type in ['SimpleRNN', 'GRU', 'LSTM']:
        print(f"Evaluating {model_type} model for Material ID: {material_id}")

        # Build and train the model
        model = build_and_train_model(X_train, y_train, model_type)

        # Forecast for the next time id (Time id 37)
        forecast_scaled = model.predict(X_train[-1:])  # Use the last time step of the training data for forecasting
        forecast = scaler.inverse_transform(forecast_scaled.reshape(-1, 1)).flatten()[0]

        # Get actual Quantity value for Time id 37
        actual_quantity = test_data['Quantity'].values[0]

        # Evaluate the model for the current Material Id
        mape = evaluate_model(np.array([forecast]), np.array([actual_quantity]))
        print(f"MAPE: {mape}")

        # Append to forecast DataFrame
        forecast_df = forecast_df.append({
            'Material_Id': material_id,
            'Model': model_type,
            'MAPE': mape
        }, ignore_index=True)

# Display the forecast DataFrame with MAPE values
print(forecast_df)


Evaluating models for Material ID: 108
Evaluating SimpleRNN model for Material ID: 108


C:\Users\sohan\anaconda3\lib\site-packages\keras\src\layers\rnn\rnn.py:205: UserWarning: Do not pass an `input_shape`/`input_dim` argument to a layer. When using Sequential models, prefer using an `Input(shape)` object as the first layer in the model instead.
  super().__init__(**kwargs)


1/1 ━━━━━━━━━━━━━━━━━━━━ 0s 173ms/step
MAPE: 14.246622721354166
Evaluating GRU model for Material ID: 108


C:\Users\sohan\AppData\Local\Temp\ipykernel_24440\1359943296.py:77: FutureWarning: The frame.append method is deprecated and will be removed from pandas in a future version. Use pandas.concat instead.
  forecast_df = forecast_df.append({
C:\Users\sohan\anaconda3\lib\site-packages\keras\src\layers\rnn\rnn.py:205: UserWarning: Do not pass an `input_shape`/`input_dim` argument to a layer. When using Sequential models, prefer using an `Input(shape)` object as the first layer in the model instead.
  super().__init__(**kwargs)


1/1 ━━━━━━━━━━━━━━━━━━━━ 0s 293ms/step
MAPE: 8.543594678243
Evaluating LSTM model for Material ID: 108


...

C:\Users\sohan\AppData\Local\Temp\ipykernel_24440\1359943296.py:77: FutureWarning: The frame.append method is deprecated and will be removed from pandas in a future version. Use pandas.concat instead.
  forecast_df = forecast_df.append({
C:\Users\sohan\anaconda3\lib\site-packages\keras\src\layers\rnn\rnn.py:205: UserWarning: Do not pass an `input_shape`/`input_dim` argument to a layer. When using Sequential models, prefer using an `Input(shape)` object as the first layer in the model instead.
  super().__init__(**kwargs)


1/1 ━━━━━━━━━━━━━━━━━━━━ 0s 261ms/step
MAPE: 246.3902791341146
Evaluating LSTM model for Material ID: 14448


C:\Users\sohan\AppData\Local\Temp\ipykernel_24440\1359943296.py:77: FutureWarning: The frame.append method is deprecated and will be removed from pandas in a future version. Use pandas.concat instead.
  forecast_df = forecast_df.append({
C:\Users\sohan\anaconda3\lib\site-packages\keras\src\layers\rnn\rnn.py:205: UserWarning: Do not pass an `input_shape`/`input_dim` argument to a layer. When using Sequential models, prefer using an `Input(shape)` object as the first layer in the model instead.
  super().__init__(**kwargs)


1/1 ━━━━━━━━━━━━━━━━━━━━ 0s 231ms/step
MAPE: 246.429443359375
   Material_Id      Model        MAPE
0          108  SimpleRNN   14.246623
1          108        GRU    8.543595
2          108       LSTM   19.490711
3          719  SimpleRNN   22.110476
4          719        GRU    2.854000
..         ...        ...         ...
64       14447        GRU  247.060585
65       14447       LSTM  248.250993
66       14448  SimpleRNN  268.884023
67       14448        GRU  246.390279
68       14448       LSTM  246.429443

[69 rows x 3 columns]


C:\Users\sohan\AppData\Local\Temp\ipykernel_24440\1359943296.py:77: FutureWarning: The frame.append method is deprecated and will be removed from pandas in a future version. Use pandas.concat instead.
  forecast_df = forecast_df.append({


In [11]:
# Calculate the overall MAPE for each model type
overall_mape = forecast_df.groupby('Model')['MAPE'].mean().reset_index()

# Print the overall MAPE for each model type
print("Overall MAPE for each model type:")
print(overall_mape)


Overall MAPE for each model type:
       Model       MAPE
0        GRU  66.661393
1       LSTM  70.535946
2  SimpleRNN  68.376284
